# Slow Grower 

In [3]:
import os
import yfinance as yf
import pandas as pd

def get_net_income(tickers):
    net_income_data = {}
    for ticker in tickers:
        try:
            # Fetch the financials data for the ticker
            fin_data = yf.Ticker(ticker).financials
            if fin_data is not None and 'Net Income' in fin_data.index:
                # Get the net income for the last 20 years
                net_income = fin_data.loc['Net Income'].iloc[-20:]
                # Extract the year from the index and group by year
                net_income.index = pd.to_datetime(net_income.index)
                net_income = net_income.groupby(net_income.index.year).sum()
                net_income_data[ticker] = net_income
            else:
                print(f"No 'Net Income' data available for {ticker}")
        except Exception as e:
            # If there's an error fetching data, skip the ticker and print the error
            print(f"Error fetching data for {ticker}: {e}")
    return net_income_data

# Define the output file path
output_file_path = 'Inputs/net_income_data_grouped.csv'

# Check if the output file already exists
if os.path.exists(output_file_path):
    print(f"Data file '{output_file_path}' already exists. Loading data from file.")
    df = pd.read_csv(output_file_path, index_col=0)
else:
    # Load tickers from the CSV file
    wilshire_df = pd.read_csv('Wilkshire_5000.csv')
    tickers = wilshire_df['Ticker'].tolist()

    # Get net income data for all tickers
    net_income_data = get_net_income(tickers)

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(net_income_data)

    # Transpose the DataFrame to have tickers as columns
    df = df.T

    # Rename columns with year_NI format
    df.columns = [f"{year}_NI" for year in df.columns]

    # Save the DataFrame to a CSV file in the specified path
    df.to_csv(output_file_path)

    print(f"Net Income Data saved to '{output_file_path}'.")


No 'Net Income' data available for AAWW
No 'Net Income' data available for AAXN
No 'Net Income' data available for ABC
No 'Net Income' data available for ABTX
No 'Net Income' data available for ACBI
No 'Net Income' data available for ACC
No 'Net Income' data available for ACER
No 'Net Income' data available for ACIA
No 'Net Income' data available for ACRX


KeyboardInterrupt: 

In [4]:
# List of Ticker and last 3 Year DIV
import yfinance as yf
import pandas as pd

def get_dividends(tickers):
    dividend_data = {}
    for ticker in tickers:
        try:
            # Fetch dividend data for the ticker
            div_data = yf.Ticker(ticker).dividends
            # Filter dividends for the years 2020-2023
            div_data_filtered = div_data.loc['2020':'2023']
            # Calculate the total amount of dividends paid for each year
            total_dividends = div_data_filtered.groupby(div_data_filtered.index.year).sum()
            dividend_data[ticker] = total_dividends
        except Exception as e:
            # If there's an error fetching data, skip the ticker and print the error
            print(f"Error fetching dividend data for {ticker}: {e}")
    return dividend_data

# Load tickers from the CSV file
wilshire_df = pd.read_csv('Wilkshire_5000.csv')
tickers = wilshire_df['Ticker'].tolist()

# Get dividend data for all tickers
dividend_data = get_dividends(tickers)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(dividend_data)

# Transpose the DataFrame to have years as columns
df = df.T

# Rename columns with year_div format
df.columns = [f"{year}_Div" for year in df.columns]

# Save the DataFrame to a CSV file
df.to_csv('dividend_data_grouped.csv')

print("Dividend Data saved to 'dividend_data_grouped.csv'.")


AAWW: No timezone found, symbol may be delisted


Error fetching dividend data for AAWW: 'RangeIndex' object has no attribute 'year'


AAXN: No timezone found, symbol may be delisted


Error fetching dividend data for AAXN: 'RangeIndex' object has no attribute 'year'


ABC: No timezone found, symbol may be delisted


Error fetching dividend data for ABC: 'RangeIndex' object has no attribute 'year'


KeyboardInterrupt: 

In [6]:
# List of Ticker and merged NI and DIV last 4 years and lists dividend growth 
import pandas as pd

# Load net income data
net_income_df = pd.read_csv('net_income_data_grouped.csv', index_col=0)

# Load dividend data
dividend_df = pd.read_csv('dividend_data_grouped.csv', index_col=0)

# Merge dataframes on index (tickers)
merged_df = pd.merge(net_income_df, dividend_df, left_index=True, right_index=True)

# Calculate dividend growth rates for consecutive years
for year in range(2020, 2023):
    current_col = f"{year}_Div"
    next_col = f"{year+1}_Div"
    growth_col = f"{year}_{year+1}_Div_Grow"
    merged_df[growth_col] = (merged_df[next_col] - merged_df[current_col]) / merged_df[current_col]

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_data.csv')

print("Merged Data saved to 'merged_data.csv'.")


Merged Data saved to 'merged_data.csv'.


# Stalwart

In [ ]:
import pandas as pd

# Read data from CSV file
net_income_df = pd.read_csv("net_income_data_grouped.csv")

# Drop the '2024_NI' column
net_income_df.drop(columns=['2024_NI'], inplace=True)

# Calculate net income growth as a percentage for each specified year
net_income_df['2020_2021_NI_Grow'] = ((net_income_df['2021_NI'] - net_income_df['2020_NI']) / net_income_df['2020_NI']) * 100
net_income_df['2021_2022_NI_Grow'] = ((net_income_df['2022_NI'] - net_income_df['2021_NI']) / net_income_df['2021_NI']) * 100
net_income_df['2022_2023_NI_Grow'] = ((net_income_df['2023_NI'] - net_income_df['2022_NI']) / net_income_df['2022_NI']) * 100

# Round the percentage growth to 4 decimal places
net_income_df = net_income_df.round({'2020_2021_NI_Grow': 4, '2021_2022_NI_Grow': 4, '2022_2023_NI_Grow': 4})

# Save the modified DataFrame to a new CSV file with the first column labeled as 'Ticker'
net_income_df.to_csv("modified_net_income_data.csv", index=False, columns=['Ticker'] + list(net_income_df.columns[1:]))

print("Modified data saved to modified_net_income_data.csv")



In [10]:
# Creates a csv file for PE ratio
import pandas as pd
import yfinance as yf

# Read the dataset
data = pd.read_csv("modified_net_income_data.csv")

# Function to get forward PE ratio from Yahoo Finance
def get_forward_pe(ticker):
    try:
        # Fetch ticker data from Yahoo Finance
        ticker_data = yf.Ticker(ticker)
        # Get forward PE ratio
        forward_pe = ticker_data.info['forwardPE']
        return forward_pe
    except:
        return None

# Apply the function to create the new column
data['2023_PE'] = data['Ticker'].apply(get_forward_pe)

# Create a new DataFrame with only 'Ticker' and '2023_PE' columns
result_df = data[['Ticker', '2023_PE']]

# Save the new DataFrame to a CSV file
result_df.to_csv("ticker_2023_PE.csv", index=False)

# Print a message to confirm the save
print("CSV file 'ticker_2023_PE.csv' has been saved successfully.")

KeyError: 'Ticker'

In [ ]:
# Creates a csv file for GSector 
import pandas as pd
import yfinance as yf

# Read the dataset
data = pd.read_csv("modified_net_income_data.csv")

# Function to get GICS sector from Yahoo Finance
def get_gsector(ticker):
    try:
        # Fetch ticker data from Yahoo Finance
        ticker_data = yf.Ticker(ticker)
        # Get GICS sector
        gsector = ticker_data.info['sector']
        return gsector
    except:
        return None

# Apply the function to create the new column
data['Gsector'] = data['Ticker'].apply(get_gsector)

# Create a new DataFrame with only 'Ticker' and 'Gsector' columns
result_df = data[['Ticker', 'Gsector']]

# Save the new DataFrame to a CSV file
result_df.to_csv("ticker_Gsector.csv", index=False)

# Print a message to confirm the save
print("CSV file 'ticker_Gsector.csv' has been saved successfully.")


In [ ]:
# Creates a csv file for Market Cap
import pandas as pd
import yfinance as yf

# Read the dataset
data = pd.read_csv("modified_net_income_data.csv")

# Function to get market capitalization from Yahoo Finance
def get_market_cap(ticker):
    try:
        # Fetch ticker data from Yahoo Finance
        ticker_data = yf.Ticker(ticker)
        # Get market cap
        market_cap = ticker_data.info['marketCap']
        return market_cap
    except:
        return None

# Apply the function to create the new column
data['2023_MarketCap'] = data['Ticker'].apply(get_market_cap)

# Create a new DataFrame with only 'Ticker' and '2023_MarketCap' columns
result_df = data[['Ticker', '2023_MarketCap']]

# Save the new DataFrame to a CSV file
result_df.to_csv("ticker_2023_MarketCap.csv", index=False)

# Print a message to confirm the save
print("CSV file 'ticker_2023_MarketCap.csv' has been saved successfully.")


In [ ]:
# Creates a data frame that calcuates the average PE ratio by Gsector 
import pandas as pd

# Load the CSV file with comma as delimiter
df = pd.read_csv("PE_Gsector_with_avg.csv")

# Remove rows with "inf" values in 2023_PE column
df = df[df['2023_PE'] != float('inf')]

# Calculate average PE ratio for each sector
avg_pe_by_sector = df.groupby('Gsector')['2023_PE'].mean()

# Fill in the Avg_PE_Gsector column based on the calculated averages
df['Avg_PE_Gsector'] = df['Gsector'].map(avg_pe_by_sector)

# Save the cleaned DataFrame back to CSV
df.to_csv("cleaned_PE_Gsector_with_avg.csv", index=False)


In [ ]:
# Creates a data frame that labels based on market cap size 
import pandas as pd

# Read the CSV file
data = pd.read_csv("ticker_2023_MarketCap.csv")

# Define the market cap size categories
categories = {
    'mega-cap': lambda x: x >= 200000000000,
    'large-cap': lambda x: 10000000000 <= x < 200000000000,
    'mid-cap': lambda x: 2000000000 <= x < 10000000000,
    'small-cap': lambda x: 250000000 <= x < 2000000000,
    'micro-cap': lambda x: x < 250000000
}

# Function to categorize market cap
def categorize_market_cap(market_cap):
    for category, condition in categories.items():
        if condition(market_cap):
            return category
    return 'Other'

# Apply categorization to the market cap column and create a new column for the category
data['Market_Cap_Cat'] = data['2023_MarketCap'].apply(categorize_market_cap)

# Save the DataFrame with the new column to a new CSV file
data.to_csv("ticker_2023_MarketCap_with_Category.csv", index=False)

print("New CSV file saved successfully!")


In [ ]:
# combines the "ticker_2023_MarketCap_with_Category.csv" , "cleaned_PE_Gsector_with_avg.csv" and ""modified_net_income_data.csv"

import pandas as pd

# Read the CSV files
market_cap_data = pd.read_csv("ticker_2023_MarketCap_with_Category.csv")
pe_gsector_data = pd.read_csv("cleaned_PE_Gsector_with_avg.csv")
net_income_data = pd.read_csv("modified_net_income_data.csv")

# Rename the first column in net_income_data to "Ticker"
net_income_data.rename(columns={net_income_data.columns[0]: 'Ticker'}, inplace=True)

# Merge the CSV files based on the 'Ticker' column
merged_data = pd.merge(market_cap_data, pe_gsector_data, on='Ticker', how='outer')
merged_data = pd.merge(merged_data, net_income_data, on='Ticker', how='outer')

# Save the merged data to a new CSV file
merged_data.to_csv("Stalwart_Master.csv", index=False)

print("Merged CSV file saved successfully!")

# Fast grower 

In [13]:
# creates a csv that combines the Net income growth and PE ratios 
import pandas as pd

# Read the modified_net_income_data.csv
net_income_df = pd.read_csv("modified_net_income_data.csv")

# Rename the first column to 'Ticker'
net_income_df.rename(columns={net_income_df.columns[0]: 'Ticker'}, inplace=True)

# Remove columns 2020_NI, 2021_NI, 2022_NI, 2023_NI
net_income_df.drop(columns=['2020_NI', '2021_NI', '2022_NI', '2023_NI'], inplace=True)

# Read the ticker_2023_PE.csv
pe_df = pd.read_csv("ticker_2023_PE.csv")

# Merge the two dataframes on the 'Ticker' column
merged_df = pd.merge(net_income_df, pe_df, on='Ticker')

# Rename the 2023_PE column
merged_df.rename(columns={'PE': '2023_PE'}, inplace=True)

# Save the merged dataframe to a new CSV file
merged_df.to_csv("NI_Grow_PE.csv", index=False)

In [14]:
# reades the created file and categoriezes based on ideal growth rate 
import pandas as pd

# Read the CSV file
df = pd.read_csv("NI_Grow_PE.csv")

# Calculate average growth rate for each row
df['Average_Growth'] = df[['2020_2021_NI_Grow', '2021_2022_NI_Grow', '2022_2023_NI_Grow']].mean(axis=1)

# Categorize the growth rate
def categorize_growth(average_growth):
    if 20 <= average_growth <= 25:
        return 'Ideal 20% - 25%'
    else:
        return 'Not Ideal'

df['Growth_Rate'] = df['Average_Growth'].apply(categorize_growth)

# Save the updated CSV with the name "Fast_Grower_Master.csv"
df.to_csv("Fast_Grower_Master.csv", index=False)

# EDA

In [19]:
# drops all missing values from the master csvs
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("Slow_Grow_Master.csv")

# Drop rows with any missing values
df_cleaned = df.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("Clean_Slow_Grow_Master.csv", index=False)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("Fast_Grower_Master.csv")

# Drop rows with any missing values
df_cleaned = df.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("Clean_Fast_Grower_Master.csv", index=False)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("Stalwart_Master.csv")

# Drop rows with any missing values
df_cleaned = df.dropna()

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("Clean_Stalwart_Master.csv", index=False)



# Screen Slow Growers


In [20]:
# Screens based on dividend growth 
import pandas as pd

# Load the CSV file
df = pd.read_csv("Clean_Slow_Grow_Master.csv")

# Filter companies with positive or flat growing dividends for each year
filtered_df = df[(df['2020_2021_Div_Grow'] >= 0) & 
                 (df['2021_2022_Div_Grow'] >= 0) & 
                 (df['2022_2023_Div_Grow'] >= 0)]

# Save the filtered data to a new CSV file
filtered_df.to_csv("Screened_Slow_Grow.csv", index=False)


# Screen Stalwarts

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("Clean_Stalwart_Master.csv")

# Filter companies with Market_Cap_Cat as 'mega cap' or 'large cap'
filtered_df = df[df['Market_Cap_Cat'].isin(['mega-cap', 'large-cap'])]

# Calculate average Net Income growth across the three years
df['Avg_NI_Growth'] = df[['2020_2021_NI_Grow', '2021_2022_NI_Grow', '2022_2023_NI_Grow']].mean(axis=1)

# Filter companies where average Net Income growth is between + or - 3% from the industry PE ratio
filtered_df = filtered_df[(df['Avg_NI_Growth'] >= 0.97 * df['Avg_PE_Gsector']) & 
                          (df['Avg_NI_Growth'] <= 1.03 * df['Avg_PE_Gsector'])]

# Add a new column for average NI growth
filtered_df['Avg_NI_Growth'] = filtered_df[['2020_2021_NI_Grow', '2021_2022_NI_Grow', '2022_2023_NI_Grow']].mean(axis=1)

# Save the filtered data to a new CSV file
filtered_df.to_csv("Screened_Stalwart.csv", index=False)


# Screen Fast Growers

In [21]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("Clean_Fast_Grower_Master.csv", sep="\t")

# Split the concatenated column names into individual names
column_names = df.columns[0].split(',')

# Create a new DataFrame with the split values
new_df = df.iloc[:, 0].str.split(',', expand=True)

# Assign the split column names to the new DataFrame
new_df.columns = column_names

# Convert 'Average_Growth' and '2023_PE' columns to numeric
new_df['Average_Growth'] = pd.to_numeric(new_df['Average_Growth'], errors='coerce')
new_df['2023_PE'] = pd.to_numeric(new_df['2023_PE'], errors='coerce')

# Filter rows where the average growth rate is between 20% and 25% and within +3 or -3 from the PE ratio
filtered_df = new_df[(new_df['Average_Growth'] >= 20) & 
                     (new_df['Average_Growth'] <= 25) &
                     (new_df['Average_Growth'] >= new_df['2023_PE'] - 3) & 
                     (new_df['Average_Growth'] <= new_df['2023_PE'] + 3)]

# Save the filtered data to a new CSV file
filtered_df.to_csv("Screened_Fast_Grower.csv", index=False)


# Comparative Results 

In [30]:
import yfinance as yf
import pandas as pd

def get_sp500_data():
    # Define the ticker symbol for S&P 500
    ticker_symbol = "^GSPC"
    
    # Define the start and end dates
    start_date = "2020-01-01"
    end_date = "2023-12-31"
    
    # Fetch the historical data
    sp500_data = yf.download(ticker_symbol, start=start_date, end=end_date)
    
    return sp500_data

def get_annual_closing_prices(data):
    # Resample the data to annual frequency and extract closing prices
    annual_closing_prices = data['Close'].resample('Y').last()
    
    return annual_closing_prices

# Fetch S&P 500 data
sp500_data = get_sp500_data()

# Get annual closing prices
annual_closing_prices = get_annual_closing_prices(sp500_data)

# Print the annual closing prices
print(annual_closing_prices)



[*********************100%%**********************]  1 of 1 completed

Date
2020-12-31    3756.070068
2021-12-31    4766.180176
2022-12-31    3839.500000
2023-12-31    4769.830078
Freq: A-DEC, Name: Close, dtype: float64


In [23]:
# Calculates the CAGR for the S&P over that time 
import pandas as pd

# Read the CSV file
sp500_data = pd.read_csv("S&P_Closing_Price.csv")

# Extract the "Close" column
closing_prices = sp500_data["Close"]

# Calculate CAGR
def calculate_cagr(closing_prices):
    n = len(closing_prices)
    initial_value = closing_prices.iloc[0]
    final_value = closing_prices.iloc[-1]
    
    cagr = ((final_value / initial_value) ** (1/n)) - 1
    
    return cagr

# Calculate CAGR for the provided closing prices
cagr = calculate_cagr(closing_prices)

# Round the CAGR to 4 decimal places
rounded_cagr = round(cagr, 4)

print("CAGR:", rounded_cagr)



CAGR: 0.0616


In [31]:
# Imports the Wilkshire closing values from 2020 - 2023
import yfinance as yf
import pandas as pd

def get_wilshire5000_data():
    # Define the ticker symbol for Wilshire 5000
    ticker_symbol = "^W5000"
    
    # Define the start and end dates
    start_date = "2020-01-01"
    end_date = "2023-12-31"
    
    # Fetch the historical data
    wilshire5000_data = yf.download(ticker_symbol, start=start_date, end=end_date)
    
    return wilshire5000_data

def get_annual_closing_prices(data):
    # Resample the data to annual frequency and extract closing prices
    annual_closing_prices = data['Close'].resample('Y').last()
    
    return annual_closing_prices

# Fetch Wilshire 5000 data
wilshire5000_data = get_wilshire5000_data()

# Get annual closing prices
annual_closing_prices_wilshire = get_annual_closing_prices(wilshire5000_data)

# Print the annual closing prices
print(annual_closing_prices_wilshire)



[*********************100%%**********************]  1 of 1 completed

Date
2020-12-31    39456.660156
2021-12-31    48461.160156
2022-12-31    38073.941406
2023-12-31    48295.378906
Freq: A-DEC, Name: Close, dtype: float64


In [29]:
# Calculates the CAGR for the Wilshire 5000 over that time
import pandas as pd

# Read the CSV file
wilshire_data = pd.read_csv("Wilshire5000_Closing_Price.csv")

# Extract the "Close" column
closing_prices = wilshire_data["Close"]

# Calculate CAGR
def calculate_cagr(closing_prices):
    n = len(closing_prices)
    initial_value = closing_prices.iloc[0]
    final_value = closing_prices.iloc[-1]
    
    cagr = ((final_value / initial_value) ** (1/n)) - 1
    
    return cagr

# Calculate CAGR for the provided closing prices
cagr = calculate_cagr(closing_prices)

# Round the CAGR to 4 decimal places
rounded_cagr = round(cagr, 4)

print("CAGR:", rounded_cagr)


CAGR: 0.0518


# Get closing prices from screened companies 

In [ ]:
# Slow Grower Closing Prices and market cap
import pandas as pd
import yfinance as yf

def get_closing_prices_and_market_cap_from_csv(csv_file, years):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Extract tickers from the "Ticker" column
    tickers = df['Ticker'].tolist()

    data = {}

    for year in years:
        for ticker in tickers:
            try:
                # Fetch stock data using yfinance for the specified year
                stock_data = yf.download(ticker, start=f'{year}-01-01', end=f'{year}-12-31')
                
                # Extract closing price for the last trading day of the year
                closing_price = stock_data['Close'].iloc[-1]
                
                # Fetch market cap
                ticker_info = yf.Ticker(ticker)
                market_cap = ticker_info.info['marketCap']
                
                # Prepare data structure
                if ticker not in data:
                    data[ticker] = {}

                data[ticker][f'{year}_Close'] = closing_price
                data[ticker][f'{year}_MarketCap'] = market_cap

            except Exception as e:
                print(f"Error fetching data for {ticker} in {year}: {e}")

    # Convert dictionary to DataFrame
    data_df = pd.DataFrame.from_dict(data, orient='index')
    
    # Save the DataFrame to a CSV file
    data_df.to_csv('Slow_Grow_Return_MarketCap.csv')

    return data_df

# Example usage:
csv_file = "Screened_Slow_Grow.csv"
years = [2020, 2021, 2022, 2023]
closing_prices_and_caps = get_closing_prices_and_market_cap_from_csv(csv_file, years)
print(closing_prices_and_caps)

In [ ]:
# Slow Grower market weighted returns 
import pandas as pd

def calculate_market_weighted_index(csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file, index_col=0)
    
    # Initialize a dictionary to store the index prices for each year
    index_prices = {}

    # For each year, calculate the market-weighted index price
    years = ['2020', '2021', '2022', '2023']
    for year in years:
        # Only consider columns for the specified year
        close_column = f'{year}_Close'
        cap_column = f'{year}_MarketCap'

        # Ensure the data is numeric, handling any potential non-numeric types that could arise from missing data
        df[close_column] = pd.to_numeric(df[close_column], errors='coerce')
        df[cap_column] = pd.to_numeric(df[cap_column], errors='coerce')

        # Drop rows where any of the required data is missing
        valid_data = df.dropna(subset=[close_column, cap_column])

        # Calculate total market capitalization
        total_market_cap = valid_data[cap_column].sum()

        # Calculate weighted sum of the closing prices
        weighted_sum = (valid_data[close_column] * valid_data[cap_column]).sum()

        # Calculate the market-weighted index price
        if total_market_cap > 0:  # Avoid division by zero
            index_price = weighted_sum / total_market_cap
        else:
            index_price = None
        
        # Store the index price in the dictionary
        index_prices[year] = index_price

    # Convert the dictionary to a DataFrame for better visualization and further use
    index_prices_df = pd.DataFrame.from_dict(index_prices, orient='index', columns=['Market_Weighted_Index_Price'])
    index_prices_df.index.name = 'Year'

    return index_prices_df

# Example usage
csv_file = 'Slow_Grow_Return_MarketCap.csv'
index_prices_df = calculate_market_weighted_index(csv_file)
print(index_prices_df)


In [ ]:
# Stalwart Closing Prices and Market Cap 
import pandas as pd
import yfinance as yf

def get_closing_prices_and_market_cap_from_csv(csv_file, years):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Extract tickers from the "Ticker" column
    tickers = df['Ticker'].tolist()

    data = {}

    for year in years:
        for ticker in tickers:
            try:
                # Fetch stock data using yfinance for the specified year
                stock_data = yf.download(ticker, start=f'{year}-01-01', end=f'{year}-12-31')
                
                # Extract closing price for the last trading day of the year
                closing_price = stock_data['Close'].iloc[-1]
                
                # Fetch market cap
                ticker_info = yf.Ticker(ticker)
                market_cap = ticker_info.info['marketCap']
                
                # Prepare data structure
                if ticker not in data:
                    data[ticker] = {}

                data[ticker][f'{year}_Close'] = closing_price
                data[ticker][f'{year}_MarketCap'] = market_cap

            except Exception as e:
                print(f"Error fetching data for {ticker} in {year}: {e}")

    # Convert dictionary to DataFrame
    data_df = pd.DataFrame.from_dict(data, orient='index')
    
    # Save the DataFrame to a CSV file
    data_df.to_csv('Stalwart_Return_MarketCap.csv')

    return data_df

# Example usage:
csv_file = "Screened_Stalwart.csv"
years = [2020, 2021, 2022, 2023]
closing_prices_and_caps = get_closing_prices_and_market_cap_from_csv(csv_file, years)
print(closing_prices_and_caps)


In [ ]:
#stalwart Market weighted return 
import pandas as pd

def calculate_market_weighted_index(csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file, index_col=0)
    
    # Initialize a dictionary to store the index prices for each year
    index_prices = {}

    # For each year, calculate the market-weighted index price
    years = ['2020', '2021', '2022', '2023']
    for year in years:
        # Only consider columns for the specified year
        close_column = f'{year}_Close'
        cap_column = f'{year}_MarketCap'

        # Ensure the data is numeric, handling any potential non-numeric types that could arise from missing data
        df[close_column] = pd.to_numeric(df[close_column], errors='coerce')
        df[cap_column] = pd.to_numeric(df[cap_column], errors='coerce')

        # Drop rows where any of the required data is missing
        valid_data = df.dropna(subset=[close_column, cap_column])

        # Calculate total market capitalization
        total_market_cap = valid_data[cap_column].sum()

        # Calculate weighted sum of the closing prices
        weighted_sum = (valid_data[close_column] * valid_data[cap_column]).sum()

        # Calculate the market-weighted index price
        if total_market_cap > 0:  # Avoid division by zero
            index_price = weighted_sum / total_market_cap
        else:
            index_price = None
        
        # Store the index price in the dictionary
        index_prices[year] = index_price

    # Convert the dictionary to a DataFrame for better visualization and further use
    index_prices_df = pd.DataFrame.from_dict(index_prices, orient='index', columns=['Market_Weighted_Index_Price'])
    index_prices_df.index.name = 'Year'

    return index_prices_df

# Example usage
csv_file = 'Stalwart_Return_MarketCap.csv'
index_prices_df = calculate_market_weighted_index(csv_file)
print(index_prices_df)


In [ ]:
# Fast Grower Closing Prices and Market Cap 
import pandas as pd
import yfinance as yf

def get_closing_prices_and_market_cap_from_csv(csv_file, years):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Extract tickers from the "Ticker" column
    tickers = df['Ticker'].tolist()

    data = {}

    for year in years:
        for ticker in tickers:
            try:
                # Fetch stock data using yfinance for the specified year
                stock_data = yf.download(ticker, start=f'{year}-01-01', end=f'{year}-12-31')
                
                # Extract closing price for the last trading day of the year
                closing_price = stock_data['Close'].iloc[-1]
                
                # Fetch market cap
                ticker_info = yf.Ticker(ticker)
                market_cap = ticker_info.info['marketCap']
                
                # Prepare data structure
                if ticker not in data:
                    data[ticker] = {}

                data[ticker][f'{year}_Close'] = closing_price
                data[ticker][f'{year}_MarketCap'] = market_cap

            except Exception as e:
                print(f"Error fetching data for {ticker} in {year}: {e}")

    # Convert dictionary to DataFrame
    data_df = pd.DataFrame.from_dict(data, orient='index')
    
    # Save the DataFrame to a CSV file
    data_df.to_csv('Fast_Grower_Return_MarketCap.csv')

    return data_df

# Example usage:
csv_file = "Screened_Fast_Grower.csv"
years = [2020, 2021, 2022, 2023]
closing_prices_and_caps = get_closing_prices_and_market_cap_from_csv(csv_file, years)
print(closing_prices_and_caps)


In [ ]:
# Fast Grower Market Weighted Return 
import pandas as pd

def calculate_market_weighted_index(csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file, index_col=0)
    
    # Initialize a dictionary to store the index prices for each year
    index_prices = {}

    # For each year, calculate the market-weighted index price
    years = ['2020', '2021', '2022', '2023']
    for year in years:
        # Only consider columns for the specified year
        close_column = f'{year}_Close'
        cap_column = f'{year}_MarketCap'

        # Ensure the data is numeric, handling any potential non-numeric types that could arise from missing data
        df[close_column] = pd.to_numeric(df[close_column], errors='coerce')
        df[cap_column] = pd.to_numeric(df[cap_column], errors='coerce')

        # Drop rows where any of the required data is missing
        valid_data = df.dropna(subset=[close_column, cap_column])

        # Calculate total market capitalization
        total_market_cap = valid_data[cap_column].sum()

        # Calculate weighted sum of the closing prices
        weighted_sum = (valid_data[close_column] * valid_data[cap_column]).sum()

        # Calculate the market-weighted index price
        if total_market_cap > 0:  # Avoid division by zero
            index_price = weighted_sum / total_market_cap
        else:
            index_price = None
        
        # Store the index price in the dictionary
        index_prices[year] = index_price

    # Convert the dictionary to a DataFrame for better visualization and further use
    index_prices_df = pd.DataFrame.from_dict(index_prices, orient='index', columns=['Market_Weighted_Index_Price'])
    index_prices_df.index.name = 'Year'

    return index_prices_df

# Example usage
csv_file = 'Fast_Grower_Return_MarketCap.csv'
index_prices_df = calculate_market_weighted_index(csv_file)
print(index_prices_df)


# Analysis


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the data for each index
# Data from 
data = {
    'Slow Grower': {2020: 163.438749, 2021: 222.309990, 2022: 185.809803, 2023: 266.864258},
    'Stalwart': {2020: 332.570957, 2021: 463.625380, 2022: 390.856517, 2023: 564.568406},
    'Fast Grower': {2020: 101.064051, 2021: 143.920488, 2022: 134.847909, 2023: 150.171399},
    'S&P 500': {2020: 3756.070068359375, 2021: 4766.180176, 2022: 3839.5, 2023: 4769.830078},
    'Wilshire 5000': {2020: 39456.66016, 2021: 48461.16016, 2022: 38073.94141, 2023: 48295.37891}
}

# Function to calculate CAGR
def calculate_cagr(start_value, end_value, periods):
    return (end_value / start_value) ** (1 / periods) - 1

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Normalize index values by dividing each value by the starting value (2020 value)
normalized_df = df.divide(df.iloc[0])

# Calculate CAGR for each series and add to the plot
cagr_values = {}
for column in normalized_df.columns:
    cagr = calculate_cagr(normalized_df[column].iloc[0], normalized_df[column].iloc[-1], len(normalized_df[column]) - 1)
    cagr_values[column] = f"{cagr:.2%}"

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
for column in normalized_df.columns:
    ax.plot(normalized_df.index, normalized_df[column], marker='o', label=f"{column} (CAGR: {cagr_values[column]})")

ax.set_title('Normalized Index Performance Comparison 2020-2023')
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Index Value')
ax.legend()
plt.grid(True)
plt.show()